# Reformat saved CSV data for training Upper Extremity Model
Mikey Fernandez
12/07/22

In [1]:
import pickle
import numpy as np
np.set_printoptions(linewidth=200)
import pandas as pd
from scipy import interpolate
import os
import math

In [10]:
# setup 
# joints = ['thumbPPos', 'thumbYPos', 'indexPos', 'mrpPos', 'wristRot', 'wristFlex', 'humPos', 'elbowPos']
joints = ['thumbPPos', 'indexPos', 'mrpPos', 'wristFlex']
emgChannels = [0, 1, 2, 4, 5, 7, 8, 9]
dataName = 'MF_0307'
data = '_2023'

saveName = 'pklFol'

OUTPUT_FPS = 60 # desired 60 Hz, will have to upsample

# information about the save file
baseFol = os.path.join(os.getcwd(), dataName)
dataFolders = sorted(os.listdir(baseFol))

saveFol = os.path.join(baseFol, saveName)
try:
    os.mkdir(saveFol)
except:
    print('Couldnt make saveFol')
    
print(dataFolders)

Couldnt make saveFol
['free1_2023-03-07_19:15:03', 'free2_2023-03-07_19:17:25', 'free_2023-03-07_19:14:28', 'handCloseOpen1_2023-03-07_19:04:57', 'handCloseOpen2_2023-03-07_19:05:37', 'handCloseOpen_2023-03-07_19:04:21', 'indexFlexExtend1_2023-03-07_18:52:48', 'indexFlexExtend2_2023-03-07_18:53:53', 'indexFlexExtend_2023-03-07_18:44:57', 'mrpFlexExtend1_2023-03-07_18:57:19', 'mrpFlexExtend2_2023-03-07_18:58:09', 'mrpFlexExtend_2023-03-07_18:56:42', 'pinch1_2023-03-07_19:10:18', 'pinch2_2023-03-07_19:11:02', 'pinch_2023-03-07_19:09:08', 'pklFol', 'pointing1_2023-03-07_19:07:49', 'pointing2_2023-03-07_19:08:29', 'pointing_2023-03-07_19:07:07', 'thumbFlexExtend1_2023-03-07_19:00:19', 'thumbFlexExtend2_2023-03-07_19:01:08', 'thumbFlexExtend_2023-03-07_18:59:00', 'wristDigits1_2023-03-07_19:12:45', 'wristDigits2_2023-03-07_19:13:25', 'wristDigits_2023-03-07_19:12:10', 'wristFlexExtend1_2023-03-07_18:46:44', 'wristFlexExtend2_2023-03-07_18:48:02', 'wristFlexExtend_2023-03-07_18:39:04']


In [11]:
def interpolateData(emg, pos, Fs_in, Fs_out=60):
    assert emg.shape[0] == pos.shape[0], 'Input data not the same size - check me!!'

    dataLen = emg.shape[0]
    t = np.arange(dataLen)/Fs_in
    upSampleRatio = Fs_out/Fs_in
    tOut = np.arange(dataLen*upSampleRatio)/Fs_out

    femgInterp = interpolate.interp1d(t, emg, axis=0, fill_value='extrapolate')
    fposInterp = interpolate.interp1d(t, pos, axis=0, fill_value='extrapolate')

    outputEMG = femgInterp(tOut)
    outputPos = fposInterp(tOut)

    return (outputEMG, outputPos)

def smoothData(pos, Fs=60, timeAverage=0.25):
    """Smooth out pos data over time using a timeAverage second window"""
    dataPointsSmoothing = int(timeAverage*Fs)
    numJoints = pos.shape[1]
    numDataPoints = pos.shape[0]

    outputPos = np.zeros_like(pos)
    for i in range(numJoints):
        outputPos[:, i] = np.convolve(pos[:, i], np.ones(dataPointsSmoothing), 'same')/dataPointsSmoothing

    return outputPos

In [18]:
for dataFol in dataFolders:
    if dataFol == saveName: continue
    
    print(f'Processing {dataFol}...')
    fullName = os.path.join(baseFol, dataFol)

    trialName = dataFol.split('_')[0]

    # load the data
    rawEMG = pd.read_csv(fullName + '/' + trialName + '_emg.csv', delimiter='\t')
    rawPos = pd.read_csv(fullName + '/' + trialName + '_pos.csv', delimiter='\t')
    with open(fullName + '/' + trialName + '_fps.txt', 'r') as file:
        inputFPS = float(file.readline())

    # get the right columns
    inputEMG = rawEMG[[str(i) for i in emgChannels]].to_numpy()
    inputPos = rawPos[joints].to_numpy()

    # resample the data at 60 Hz for use to train the
    (outputEMG, outputPos) = interpolateData(inputEMG, inputPos, inputFPS, OUTPUT_FPS)

    # smooth data
    smoothedPos = smoothData(outputPos)
    smoothedDf = pd.DataFrame(data=smoothedPos, columns=joints)
    smoothedDf.to_csv(fullName + '/' + trialName + '_smoothedPos.csv', encoding='utf-8', sep="\t", index=False)

    # put the data in the expected form
    timeSteps = outputPos.shape[0]
    # startRange = 0 if startTime < 0 else math.ceil(startTime*OUTPUT_FPS)
    # endRange = timeSteps if endTime < 0 else math.ceil(endTime*OUTPUT_FPS)

    posPkl = []
    emgPkl = []
    smoothPkl = []
    # for t in range(startRange, endRange):
    for t in range(timeSteps):
        posPkl.append(np.array(list(outputPos[t, :])))
        emgPkl.append(np.array(list(outputEMG[t, :])))
        smoothPkl.append(np.array(list(smoothedPos[t, :])))

    pklData = [emgPkl, posPkl, smoothPkl]

    # save this into a pkl file
    with open(saveFol + '/' + trialName + '.pkl', 'wb') as file:
        pickle.dump(pklData, file)

    os.mv

Processing free1_2023-03-07_19:15:03...
Processing free2_2023-03-07_19:17:25...
Processing free_2023-03-07_19:14:28...
Processing handCloseOpen1_2023-03-07_19:04:57...
Processing handCloseOpen2_2023-03-07_19:05:37...
Processing handCloseOpen_2023-03-07_19:04:21...
Processing indexFlexExtend1_2023-03-07_18:52:48...
Processing indexFlexExtend2_2023-03-07_18:53:53...
Processing indexFlexExtend_2023-03-07_18:44:57...
Processing mrpFlexExtend1_2023-03-07_18:57:19...
Processing mrpFlexExtend2_2023-03-07_18:58:09...
Processing mrpFlexExtend_2023-03-07_18:56:42...
Processing pinch1_2023-03-07_19:10:18...
Processing pinch2_2023-03-07_19:11:02...
Processing pinch_2023-03-07_19:09:08...
Processing pointing1_2023-03-07_19:07:49...
Processing pointing2_2023-03-07_19:08:29...
Processing pointing_2023-03-07_19:07:07...
Processing thumbFlexExtend1_2023-03-07_19:00:19...
Processing thumbFlexExtend2_2023-03-07_19:01:08...
Processing thumbFlexExtend_2023-03-07_18:59:00...
Processing wristDigits1_2023-03-

In [1295]:
# load the data
rawEMG = pd.read_csv(dataFol + '/' + fileNames + '_emg.csv', delimiter='\t')
rawPos = pd.read_csv(dataFol + '/' + fileNames + '_pos.csv', delimiter='\t')
with open(dataFol + '/' + fileNames + '_fps.txt', 'r') as file:
    inputFPS = float(file.readline())

# get the right columns
inputEMG = rawEMG[[str(i) for i in emgChannels]].to_numpy()
inputPos = rawPos[joints].to_numpy()

In [1297]:
# resample the data at 60 Hz for use to train the
(outputEMG, outputPos) = interpolateData(inputEMG, inputPos, inputFPS, OUTPUT_FPS)

In [1298]:
# smooth data
smoothedPos = smoothData(outputPos)
smoothedDf = pd.DataFrame(data=smoothedPos, columns=joints)
smoothedDf.to_csv(dataFol + '/' + fileNames + '_smoothedPos.csv', encoding='utf-8', sep="\t", index=False)

In [1299]:
# put the data in the expected form
timeSteps = outputPos.shape[0]
startRange = 0 if startTime < 0 else math.ceil(startTime*OUTPUT_FPS)
endRange = timeSteps if endTime < 0 else math.ceil(endTime*OUTPUT_FPS)

posPkl = []
emgPkl = []
smoothPkl = []
for t in range(startRange, endRange):
    outputPos[t, 0:4] = np.pi/4
    smoothedPos[t, 0:4] = np.pi/4
    outputPos[t, 4] -= 0.23
    smoothedPos[t, 4] -= 0.23
    posPkl.append(np.array(list(outputPos[t, :])))
    emgPkl.append(np.array(list(outputEMG[t, :])))
    smoothPkl.append(np.array(list(smoothedPos[t, :])))


pklData = [emgPkl, posPkl, smoothPkl]

In [1304]:
# save this into a pkl file
with open(dataFol + '/' + fileNames + '_pkl.pkl', 'wb') as file:
    pickle.dump(pklData, file)

In [21]:
import matplotlib.pyplot as plt
%matplotlib qt

testFol = os.path.join(saveFol)
fileName = 'free2'

with open(testFol + '/' + fileName + '.pkl', 'rb') as file:
    loadedData = pickle.load(file)

E = loadedData[0]
T = loadedData[1]
T_smooth = loadedData[2]

timeSteps = np.array(E).shape[0]
numEMG = np.array(E).shape[1]
numJoints = np.array(T).shape[1]
t = np.linspace(0, timeSteps - 1, timeSteps)/OUTPUT_FPS

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1)
fig.suptitle(fileName)
for i in range(numEMG):
    ax1.plot(t, np.array(E)[:, i], label=f'{emgChannels[i]}')

ax1.autoscale(enable=True, axis='x', tight=True)
ax1.legend()

for i in range(numJoints):
    ax2.plot(t, np.array(T)[:, i], label=f'{joints[i]}')
plt.gca().set_prop_cycle(None)

for i in range(numJoints):
    ax2.plot(t, np.array(T_smooth)[:, i], label=f'{joints[i]} Smooth', linestyle='dashed')

ax2.autoscale(enable=True, axis='x', tight=True)
ax2.legend()